In [1]:
!nvidia-smi

Mon Jan  8 13:51:20 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/drive/MyDrive/DD2360\ Applied\ GPU/project/cuda/backprop/

/content/drive/MyDrive/DD2360 Applied GPU/project/cuda/backprop


In [81]:
# change the Makefile line 25 to $(CC) $(CC_FLAGS) -no-pie backprop.o facetrain.o imagenet.o backprop_cuda.o -o backprop -L$(CUDA_LIB_DIR) -lcuda -lcudart -lm
! make backprop

/usr/local/cuda/bin/nvcc -I/usr/local/cuda/include -O2 -c backprop_cuda.cu
gcc -g -O2 -no-pie backprop.o facetrain.o imagenet.o backprop_cuda.o -o backprop -L/usr/local/cuda/lib64 -lcuda -lcudart -lm 


In [82]:
! nvprof ./backprop 65536

Random number generator seed: 7
==46950== NVPROF is profiling process 46950, command: ./backprop 65536
Input layer size : 65536
Starting training kernel
Performing unified memory GPU computation
epoch 0: out_err = 0.000030
Performing unified memory GPU computation
epoch 1: out_err = 0.000030
Performing unified memory GPU computation
epoch 2: out_err = 0.000030
Performing unified memory GPU computation
epoch 3: out_err = 0.000030
Performing unified memory GPU computation
epoch 4: out_err = 0.000030
Performing unified memory GPU computation
epoch 5: out_err = 0.000031
Performing unified memory GPU computation
epoch 6: out_err = 0.000031
Performing unified memory GPU computation
epoch 7: out_err = 0.000031
Performing unified memory GPU computation
epoch 8: out_err = 0.000031
Performing unified memory GPU computation
epoch 9: out_err = 0.000031
Performing unified memory GPU computation
epoch 10: out_err = 0.000031
Performing unified memory GPU computation
epoch 11: out_err = 0.000031
Perfo

In [ ]:
! nvprof ./backprop 65536

Random number generator seed: 7
Input layer size : 65536
Starting training kernel
==5938== NVPROF is profiling process 5938, command: ./backprop 65536
Performing GPU computation
Training done
==5938== Profiling application: ./backprop 65536
==5938== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   69.30%  2.3551ms         5  471.01us  1.1520us  778.32us  [CUDA memcpy HtoD]
                   15.42%  523.89us         3  174.63us  21.696us  479.51us  [CUDA memcpy DtoH]
                   12.13%  412.31us         1  412.31us  412.31us  412.31us  bpnn_adjust_weights_cuda(float*, int, float*, int, float*, float*)
                    3.16%  107.26us         1  107.26us  107.26us  107.26us  bpnn_layerforward_CUDA(float*, float*, float*, float*, int, int)
      API calls:   94.21%  93.309ms         6  15.552ms  4.5140us  93.007ms  cudaMalloc
                    4.47%  4.4314ms         8  553.93us  14.600us  1.0001ms  cudaMem

In [ ]:
! nvprof ./backprop 65536 # stream version

Random number generator seed: 7
Input layer size : 65536
Starting training kernel
==2982== NVPROF is profiling process 2982, command: ./backprop 65536
Performing stream GPU computation
Training done
==2982== Profiling application: ./backprop 65536
==2982== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   47.28%  1.4479ms       129  11.224us     672ns  15.552us  [CUDA memcpy HtoD]
                   25.65%  785.69us         3  261.90us  2.5600us  761.28us  [CUDA memcpy DtoH]
                   20.20%  618.65us        32  19.332us  19.232us  19.456us  bpnn_adjust_weights_cuda(float*, int, float*, int, float*, float*)
                    6.87%  210.49us        32  6.5770us  6.4640us  6.8160us  bpnn_layerforward_CUDA(float*, float*, float*, float*, int, int)
      API calls:   96.50%  178.56ms         6  29.761ms  4.7100us  178.34ms  cudaMalloc
                    2.44%  4.5150ms       256  17.636us     714ns  1.0586ms  

In [ ]:
! nvprof ./backprop 65536 # stream version cudafree

Random number generator seed: 7
Input layer size : 65536
Starting training kernel
==1882== NVPROF is profiling process 1882, command: ./backprop 65536
Performing stream GPU computation
Training done
==1882== Profiling application: ./backprop 65536
==1882== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   46.50%  1.4377ms       129  11.145us     704ns  16.960us  [CUDA memcpy HtoD]
                   26.66%  824.34us         3  274.78us  2.5600us  799.93us  [CUDA memcpy DtoH]
                   20.01%  618.72us        32  19.334us  19.136us  19.520us  bpnn_adjust_weights_cuda(float*, int, float*, int, float*, float*)
                    6.82%  210.94us        32  6.5910us  6.5280us  6.8800us  bpnn_layerforward_CUDA(float*, float*, float*, float*, int, int)
      API calls:   93.31%  86.738ms         7  12.391ms  4.0270us  86.015ms  cudaFree
                    4.97%  4.6177ms       256  18.037us     727ns  1.0743ms  cu

In [ ]:
! nvprof --export-profile stream.nvprof -f ./backprop 65536

Random number generator seed: 7
Input layer size : 65536
Starting training kernel
==1036== NVPROF is profiling process 1036, command: ./backprop 65536
Performing stream GPU computation
Training done
==1036== Generated result file: /content/drive/.shortcut-targets-by-id/17G7OsCBYj_LsEnCLIXEBeINsnY8-7Icj/DD2360 Applied GPU/project/cuda/backprop/stream.nvprof


In [ ]:
! nvprof --export-profile nonstream.nvprof -f ./backprop 65536

Random number generator seed: 7
Input layer size : 65536
Starting training kernel
==6179== NVPROF is profiling process 6179, command: ./backprop 65536
Performing GPU computation
Training done
==6179== Generated result file: /content/drive/.shortcut-targets-by-id/17G7OsCBYj_LsEnCLIXEBeINsnY8-7Icj/DD2360 Applied GPU/project/cuda/backprop/nonstream.nvprof


In [ ]:
! nvprof --export-profile streamfree.nvprof -f ./backprop 65536

Random number generator seed: 7
Input layer size : 65536
Starting training kernel
==2235== NVPROF is profiling process 2235, command: ./backprop 65536
Performing stream GPU computation
Training done
==2235== Generated result file: /content/drive/.shortcut-targets-by-id/17G7OsCBYj_LsEnCLIXEBeINsnY8-7Icj/DD2360 Applied GPU/project/cuda/backprop/streamfree.nvprof


In [84]:
! nvprof --export-profile um1000prefetch.nvprof -f ./backprop 65536

Random number generator seed: 7
==50962== NVPROF is profiling process 50962, command: ./backprop 65536
Input layer size : 65536
Starting training kernel
Performing unified memory GPU computation
epoch 0: out_err = 0.000030
Performing unified memory GPU computation
epoch 1: out_err = 0.000030
Performing unified memory GPU computation
epoch 2: out_err = 0.000030
Performing unified memory GPU computation
epoch 3: out_err = 0.000030
Performing unified memory GPU computation
epoch 4: out_err = 0.000030
Performing unified memory GPU computation
epoch 5: out_err = 0.000031
Performing unified memory GPU computation
epoch 6: out_err = 0.000031
Performing unified memory GPU computation
epoch 7: out_err = 0.000031
Performing unified memory GPU computation
epoch 8: out_err = 0.000031
Performing unified memory GPU computation
epoch 9: out_err = 0.000031
Performing unified memory GPU computation
epoch 10: out_err = 0.000031
Performing unified memory GPU computation
epoch 11: out_err = 0.000031
Perfo

In [83]:
! nvprof --export-profile gpu1000.nvprof -f ./backprop 65536

Random number generator seed: 7
==50566== NVPROF is profiling process 50566, command: ./backprop 65536
Input layer size : 65536
Starting training kernel
Performing unified memory GPU computation
epoch 0: out_err = 0.000030
Performing unified memory GPU computation
epoch 1: out_err = 0.000030
Performing unified memory GPU computation
epoch 2: out_err = 0.000030
Performing unified memory GPU computation
epoch 3: out_err = 0.000030
Performing unified memory GPU computation
epoch 4: out_err = 0.000030
Performing unified memory GPU computation
epoch 5: out_err = 0.000031
Performing unified memory GPU computation
epoch 6: out_err = 0.000031
Performing unified memory GPU computation
epoch 7: out_err = 0.000031
Performing unified memory GPU computation
epoch 8: out_err = 0.000031
Performing unified memory GPU computation
epoch 9: out_err = 0.000031
Performing unified memory GPU computation
epoch 10: out_err = 0.000031
Performing unified memory GPU computation
epoch 11: out_err = 0.000031
Perfo

In [41]:
! nvprof --export-profile PM1000.nvprof -f ./backprop 65536

Random number generator seed: 7
Input layer size : 65536
Starting training kernel
==20497== NVPROF is profiling process 20497, command: ./backprop 65536
Performing pin memory computation
epoch 0: out_err = 0.000030
Performing pin memory computation
epoch 1: out_err = 0.000030
Performing pin memory computation
epoch 2: out_err = 0.000030
Performing pin memory computation
epoch 3: out_err = 0.000030
Performing pin memory computation
epoch 4: out_err = 0.000030
Performing pin memory computation
epoch 5: out_err = 0.000031
Performing pin memory computation
epoch 6: out_err = 0.000031
Performing pin memory computation
epoch 7: out_err = 0.000031
Performing pin memory computation
epoch 8: out_err = 0.000031
Performing pin memory computation
epoch 9: out_err = 0.000031
Performing pin memory computation
epoch 10: out_err = 0.000031
Performing pin memory computation
epoch 11: out_err = 0.000031
Performing pin memory computation
epoch 12: out_err = 0.000031
Performing pin memory computation
epoc

In [7]:
! make clean backprop

rm -f *.o *~ backprop backprop_cuda.linkinfo
gcc -g -O2 backprop.c -c
backprop.c: In function ‘gettime’:
backprop.c:32:3: warning: implicit declaration of function ‘gettimeofday’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wimplicit-function-declaration-Wimplicit-function-declaration]8;;]
   32 |   gettimeofday(&t,NULL);
      |   ^~~~~~~~~~~~
backprop.c: In function ‘alloc_1d_dbl’:
backprop.c:69:3: warning: implicit declaration of function ‘cudaMallocManaged’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wimplicit-function-declaration-Wimplicit-function-declaration]8;;]
   69 |   cudaMallocManaged((void**)&new,(unsigned) (n * sizeof (float)));
      |   ^~~~~~~~~~~~~~~~~
backprop.c: At top level:
backprop.c:107:1: warning: return type defaults to ‘int’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wimplicit-int-Wimplicit-int]8;;]
  107 | bpnn_randomize_weights(w, m, n)
      | ^~~~~~~~~~~~~~~~~~~~~~
backprop.